# Image classification with small data

1. Download and setup dataset (Kaggle Cats & Dogs) https://www.kaggle.com/c/dogs-vs-cats/data
2. Train a small convnet on our small data (acc ~ 0.75)
3. Train a MLP using bottleneck features of pretrained model (acc ~ 0.90)
4. Finetuning top layers of pretrained model on our small data (acc ~ 0.94)


In [1]:
# imports
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K
from keras import applications
import numpy as np
from keras import Model
from keras import optimizers
import matplotlib.pyplot as plt

/Users/garethjones/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
# define plot & parse functions
def parse_log_file(pfile):
    f= open(pfile).readlines()
    lines = [i.strip() for i in f]
    log= {k:[] for k in ['acc','val_acc','loss','val_loss']}
    for l in lines:
        if 'loss' in l:
            for k in log.keys():
                log[k].append(float(l.split(k+': ')[1].split(' -')[0]))
    return log 

def plot_loss_acc(pfile):
    history = parse_log_file(pfile)
    plt.plot(history['loss'])
    plt.plot(history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'val'], loc='upper left')
    plt.show()
    plt.plot(history['acc'])
    plt.plot(history['val_acc'])
    plt.title('model acc')
    plt.ylabel('acc')
    plt.xlabel('epoch')
    plt.legend(['train', 'val'], loc='upper left')
    plt.show()

In [3]:
img_width, img_height = 150, 150
input_shape = (img_width, img_height, 3)

## Train a small convnet on our small data

In [4]:
# define a sequential model (small conv net): 
# 3 conv blocks (Conv2D, Activation('relu'), MaxPooling2D) + 2 dense layers
# Conv_1: filters 32, kernel size(3,3)
# Conv_2: filters 32, kernel size(3,3)
# Conv_3: filters 64, kernel size(3,3)
# Flatten
# Dense_1: 64
# Activation('relu')
# Dropout(0.5)
# Dense_2:  ? 
# Activation('sigmoid')

# print model summary

model = Sequential()
model.add(Conv2D(32, (3,3), activation='relu', input_shape=input_shape))
model.add(MaxPooling2D((2,2)))
model.add(Conv2D(32, (3,3), activation='relu'))
model.add(MaxPooling2D((2,2)))
model.add(Conv2D(64, (3,3), activation='relu'))
model.add(MaxPooling2D((2,2)))
model.add(Flatten())
model.add(Dense(64, activation = 'relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation = 'sigmoid'))

print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 148, 148, 32)      896       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 74, 74, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 72, 72, 32)        9248      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 36, 36, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 34, 34, 64)        18496     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 17, 17, 64)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 18496)             0         
__________

In [5]:
# compile model 
# use binary crossentropy loss 
# and rmsprop optimizer

model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy']
)

In [9]:
train_data_dir = '/Users/garethjones/Documents/Data Science/Data Science Retreat/6. Zero & Low Shot Learning - Nour/Exercises/Small CNN Classifier/data/train'
validation_data_dir = '/Users/garethjones/Documents/Data Science/Data Science Retreat/6. Zero & Low Shot Learning - Nour/Exercises/Small CNN Classifier/data/validation'
nb_train_samples = 2000
nb_validation_samples = 800
epochs = 50
batch_size = 16

In [10]:
# define a keras ImageDataGenerator for training data with appropriate augmentation 
# use rescale=1. / 255 to normalise pixles values

# define a keras ImageDataGenerator for test data  (no augmentation only rescaling)

img_train_gen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
)

img_test_gen = ImageDataGenerator(
    rescale=1./255
)

In [11]:
# define training and validation iterators 
# use ImageDataGenerator.flow_from_directory to training and validation dirs accordingly 

# # add your implementation

train_batches = img_train_gen.flow_from_directory(
    train_data_dir,
    target_size=(img_width,img_height),
    batch_size=batch_size,
    class_mode='binary'
)

test_batches = img_test_gen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width,img_height),
    batch_size=batch_size,
    class_mode='binary'
)


Found 2000 images belonging to 2 classes.
Found 800 images belonging to 2 classes.


In [35]:
# train and validate the model using fit_generator

model.fit_generator(train_batches,
                    steps_per_epoch=(nb_train_samples/batch_size),
                    epochs = 30
                   )

Epoch 1/3
125/125 [==============================] - 934s 7s/step - loss: 0.8425 - acc: 0.4900
Epoch 2/3


KeyboardInterrupt: 

## Train a MLP using bottleneck features

In [19]:
# define VGG16 network using keras applications.VGG16  
# set weights = 'imagenet'
# set include_top=False : don't include the fully-connected layer at the top of the network 
# set input_shape: image shape

from keras.applications import vgg16

model = vgg16.VGG16(
    weights = 'imagenet',
    include_top = False,
    input_shape = input_shape
)

# define a keras ImageDataGenerator for data (no augmentation only rescaling)
# define training and validation iterators same as before (set shuffle=False)

img_gen = ImageDataGenerator(
    rescale=1./255
)

train_batches = img_gen.flow_from_directory(
    train_data_dir,
    target_size=(img_width,img_height),
    batch_size=batch_size,
    shuffle = False
)

test_batches = img_gen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width,img_height),
    batch_size=batch_size,
    shuffle = False
)

model.summary()

Found 2000 images belonging to 2 classes.
Found 800 images belonging to 2 classes.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 150, 150, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 150, 150, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 150, 150, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 75, 75, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 75, 75, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 75, 75, 128)       147584    
___________________________________________________________

In [21]:
# extract image features for training and validation separately using predict_generator
# this is where we pass our images to the VGG model, and it predicts certain image features, rather than classes

train_features = model.predict_generator(train_batches,
                        steps=(nb_train_samples/batch_size),
                        verbose=1
                       )

test_features = model.predict_generator(test_batches,
                                       steps=(nb_validation_samples/batch_size),
                                        verbose=1
                                        )

  2/125 [..............................] - ETA: 4:41:41

KeyboardInterrupt: 

In [26]:
# create training and validation labels array (features are in order because we used shuffle=false) 

train_labels = []
for i in train_batches:
    imgs,labels = next(train_batches)
    train_labels.append(labels)


# define model of two dense layers 256 and ? 
# model should start with Flatten layer (to flatten extracted image features to a vector)
# don’t forget activations and dropout(0.5)
# compile with binary_crossentropy loss and rmsprop optimizer

model = Sequential()
model.add(Dense(256,activation='relu'))

# Train MLP using fit function

# # add your implementation

# save weights to use them in fine-tuning later on

# # add your implementation

KeyboardInterrupt: 

## Finetuning top layers of pretrained model on our small data

In [ ]:
# define new model:  VGG16 as base and MLP as top 
# input = base_model.input, output=top_model(base_model.output)

# # add your implementation

# freeze the first 15 layers (up to the last conv block)
# set trainable=false (weights will not be updated)

# # add your implementation

# compile with binary_crossentropy loss and SGD with low learing rate optimizer

# # add your implementation

In [ ]:
# fine-tune the model using fit_generator and train and validation iterators